# Ensemble model

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
from src.ensemble import EnsembleModel

import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

## Step 1: Load models

### Distilbert

In [15]:
distilbert_name = "jeffnjy/distilbert-base-test"
distilbert_tokenizer = AutoTokenizer.from_pretrained(distilbert_name)
distilbert_model = AutoModelForQuestionAnswering.from_pretrained(distilbert_name)

### Albert

In [16]:
albert_name = "jeffnjy/albert-base"
albert_tokenizer = AutoTokenizer.from_pretrained(albert_name)
albert_model = AutoModelForQuestionAnswering.from_pretrained(albert_name)

### Roberta

In [17]:
roberta_pretrained_name = "roberta-base"
roberta_local_path = "./tmp/model/roberta_baseline.pt"
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_pretrained_name)
roberta_model = torch.load(roberta_local_path)

### Bert

In [18]:
# bert_pretrained_name = "bert-base-uncased"
# bert_local_path = "./tmp/model/bert_baseline.pt"
# bert_tokenizer = AutoTokenizer.from_pretrained(bert_pretrained_name)
# bert_model = torch.load(bert_local_path)

In [19]:
bert_checkpoint = "./tmp/checkpoint/bert-squad-baseline-model"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_checkpoint, local_files_only=True)
bert_model = AutoModelForQuestionAnswering.from_pretrained(bert_checkpoint, local_files_only=True)

### XLNet

In [20]:
xlnet_checkpoint = "./tmp/checkpoint/XLNET"
xlnet_tokenizer = AutoTokenizer.from_pretrained(xlnet_checkpoint, local_files_only=True)
xlnet_model = AutoModelForQuestionAnswering.from_pretrained(xlnet_checkpoint, local_files_only=True)

Some weights of XLNetForQuestionAnsweringSimple were not initialized from the model checkpoint at ./tmp/checkpoint/XLNET and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Step 2: Create ensemble model

The weight of each model can be changed here

In [21]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
ensemble_model = EnsembleModel(device=device)
ensemble_model.add_model(model=distilbert_model, tokenizer=distilbert_tokenizer, weight=1.0)
ensemble_model.add_model(model=albert_model, tokenizer=albert_tokenizer, weight=1.0)
ensemble_model.add_model(model=roberta_model, tokenizer=roberta_tokenizer, weight=1.0)
ensemble_model.add_model(model=bert_model, tokenizer=bert_tokenizer, weight=1.0)
ensemble_model.add_model(model=xlnet_model, tokenizer=xlnet_tokenizer, weight=1.0)

## Step 3: Perform perdiction

In [22]:
question = "How many programming languages does BLOOM support?"
context = """BLOOM has 176.5 billion parameters and can generate text in 
46 languages natural languages and 13.37 programming languages."""

### Option a: Weighted sum of softmax probabilities

In [23]:
ensemble_model.predict(question=question, context=context, mode="soft")

'13.37'

### Option b: Weighted votes

In [24]:
ensemble_model.predict(question=question, context=context, mode="hard")

'13.37'